In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
from epiweeks import Week
from metrics import *
EPS = 1e-6
import matplotlib.pyplot as plt
import math

In [2]:
# ground truth
df_ground_truth = pd.read_csv("./data-truth/truth-incident Hospitalizations.csv")
df_ground_truth

,date,location,location_name,value
0,2020-01-11,01,Alabama,0
1,2020-01-11,15,Hawaii,0
2,2020-01-11,18,Indiana,0
3,2020-01-11,27,Minnesota,0
4,2020-01-11,30,Montana,0
...,...,...,...,...
7737,2022-11-26,50,Vermont,15
7738,2022-11-26,53,Washington,572
7739,2022-11-26,55,Wisconsin,258
7740,2022-11-26,54,West Virginia,209


In [3]:
df_ground_truth = df_ground_truth[df_ground_truth['date'] >= '2022-10-22']
df_ground_truth['epiweek'] = df_ground_truth['date'].apply(lambda x: Week.fromdate(datetime.strptime(x, '%Y-%m-%d').date()))
df_ground_truth

/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_73848/255273429.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ground_truth['epiweek'] = df_ground_truth['date'].apply(lambda x: Week.fromdate(datetime.strptime(x, '%Y-%m-%d').date()))


,date,location,location_name,value,epiweek
7418,2022-10-22,02,Alaska,3,202242
7419,2022-10-22,01,Alabama,141,202242
7420,2022-10-22,05,Arkansas,50,202242
7421,2022-10-22,04,Arizona,22,202242
7422,2022-10-22,06,California,124,202242
...,...,...,...,...,...
7737,2022-11-26,50,Vermont,15,202247
7738,2022-11-26,53,Washington,572,202247
7739,2022-11-26,55,Wisconsin,258,202247
7740,2022-11-26,54,West Virginia,209,202247


In [4]:
location_df = pd.read_csv('data-locations/locations.csv')
location_dict = {location_df['location_name'][i]:location_df['abbreviation'][i] for i in range(len(location_df)) if not pd.isnull(location_df['abbreviation'][i])}
location_dict['United States'] = 'US'
location_dict

{'US': 'US',
 'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 '

In [5]:
df_grnd = df_ground_truth
df_grnd = df_grnd.rename(
    columns = {'epiweek':"predicted_week"}
)
df_grnd['location_name'] = df_grnd['location_name'].apply(lambda x: location_dict[x])
df_grnd['predicted_week'] = df_grnd['predicted_week'].astype(str)
df_grnd

,date,location,location_name,value,predicted_week
7418,2022-10-22,02,AK,3,202242
7419,2022-10-22,01,AL,141,202242
7420,2022-10-22,05,AR,50,202242
7421,2022-10-22,04,AZ,22,202242
7422,2022-10-22,06,CA,124,202242
...,...,...,...,...,...
7737,2022-11-26,50,VT,15,202247
7738,2022-11-26,53,WA,572,202247
7739,2022-11-26,55,WI,258,202247
7740,2022-11-26,54,WV,209,202247


In [6]:
file_dir = './predictions.csv' 
df_total = pd.read_csv(file_dir)

In [7]:
df_total['model'].nunique()
df_final = df_total.copy()
all_model_names = np.array(df_final['model'].drop_duplicates())

In [8]:
all_model_names = np.array(df_final['model'].drop_duplicates())
df_gt = df_final[df_final['model']=='GT-FluFNP']

# GT-FluFNP model hasn't predicted for some locations 
all_regions = np.array(df_gt['location'].drop_duplicates())
regions_ground_truth = np.array(df_grnd['location'].drop_duplicates())

In [9]:
df_point = df_final[df_final['type']=='point']
df_quant = df_final[df_final['type']=='quantile']
df_quant

,model,forecast_week,ahead,location,type,quantile,value
1,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.010,53.337132
2,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.025,54.761291
3,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.050,77.427982
4,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.100,138.520775
5,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.150,158.317353
...,...,...,...,...,...,...,...
700437,VTSanghani-Transformer,202245,4,WY,quantile,0.850,21.307235
700438,VTSanghani-Transformer,202245,4,WY,quantile,0.900,26.563623
700439,VTSanghani-Transformer,202245,4,WY,quantile,0.950,31.820012
700440,VTSanghani-Transformer,202245,4,WY,quantile,0.975,34.448206


In [10]:
weeks = np.array(df_point['forecast_week'].drop_duplicates())
max_week = df_grnd['predicted_week'].max()
max_week

'202247'

In [11]:
df_point['predicted_week'] = df_point['forecast_week']+df_point['ahead']

# Have ground truth only till week 10  
df_point['predicted_week'] = df_point['predicted_week']
df_point['predicted_week'] = df_point['predicted_week'].astype(str)
df_point = df_point[df_point['predicted_week']<=max_week] 
df_point

/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_73848/2688124874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_point['predicted_week'] = df_point['forecast_week']+df_point['ahead']
/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_73848/2688124874.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_point['predicted_week'] = df_point['predicted_week']
/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_73848/2688124874.py:5: SettingWithCopyWarning: 
A value is trying to

,model,forecast_week,ahead,location,type,quantile,value,predicted_week
0,CADPH-FluCAT_Ensemble,202242,1,CA,point,NaN,194.806293,202243
24,CADPH-FluCAT_Ensemble,202242,2,CA,point,NaN,200.584234,202244
48,CADPH-FluCAT_Ensemble,202242,3,CA,point,NaN,221.527254,202245
72,CADPH-FluCAT_Ensemble,202242,4,CA,point,NaN,243.912329,202246
96,CADPH-FluCAT_Ensemble,202243,1,CA,point,NaN,108.111827,202244
...,...,...,...,...,...,...,...,...
700178,VTSanghani-Transformer,202245,2,WI,point,NaN,25.105520,202247
700250,VTSanghani-Transformer,202245,1,WV,point,NaN,21.550989,202246
700274,VTSanghani-Transformer,202245,2,WV,point,NaN,18.652676,202247
700346,VTSanghani-Transformer,202245,1,WY,point,NaN,6.341624,202246


In [12]:
print(df_grnd)

            date location location_name  value predicted_week
7418  2022-10-22       02            AK      3         202242
7419  2022-10-22       01            AL    141         202242
7420  2022-10-22       05            AR     50         202242
7421  2022-10-22       04            AZ     22         202242
7422  2022-10-22       06            CA    124         202242
...          ...      ...           ...    ...            ...
7737  2022-11-26       50            VT     15         202247
7738  2022-11-26       53            WA    572         202247
7739  2022-11-26       55            WI    258         202247
7740  2022-11-26       54            WV    209         202247
7741  2022-11-26       56            WY     45         202247

[324 rows x 5 columns]


In [13]:
# Merging the two datasets on predicted week
df_newpoint = pd.merge(df_point, df_grnd, on = "predicted_week")
# Removing all unnecessary merges
df_newpoint = df_newpoint[df_newpoint['location_x'] == df_newpoint['location_name']]
df_newpoint

,model,forecast_week,ahead,location_x,type,quantile,value_x,predicted_week,date,location_y,location_name,value_y
4,CADPH-FluCAT_Ensemble,202242,1,CA,point,NaN,1.948063e+02,202243,2022-10-29,06,CA,211
54,CMU-TimeSeries,202242,1,AK,point,NaN,3.731247e+00,202243,2022-10-29,02,AK,14
109,CMU-TimeSeries,202242,1,AL,point,NaN,1.561357e+02,202243,2022-10-29,01,AL,262
164,CMU-TimeSeries,202242,1,AR,point,NaN,5.610412e+01,202243,2022-10-29,05,AR,84
219,CMU-TimeSeries,202242,1,AZ,point,NaN,2.809131e+01,202243,2022-10-29,04,AZ,34
...,...,...,...,...,...,...,...,...,...,...,...,...
646267,VTSanghani-Transformer,202245,2,VT,point,NaN,3.217188e-09,202247,2022-11-26,50,VT,15
646322,VTSanghani-Transformer,202245,2,WA,point,NaN,2.162635e+01,202247,2022-11-26,53,WA,572
646377,VTSanghani-Transformer,202245,2,WI,point,NaN,2.510552e+01,202247,2022-11-26,55,WI,258
646432,VTSanghani-Transformer,202245,2,WV,point,NaN,1.865268e+01,202247,2022-11-26,54,WV,209


In [14]:
model = df_newpoint['model'].unique()
model_list = []
for i in model: 
    if(len(df_newpoint[df_newpoint['model'] == i]) > 0):
        model_list.append(i)
model_list = model_list[1:]
model_list.remove('UGuelph-FluPLUG')
model_list.remove('MOBS-GLEAM_FLUH')
model_list.remove('SigSci-CREG')
model_list.remove('ISU_NiemiLab-Flu')
model_list.remove('LUcompUncertLab-humanjudgment')
model_list.remove('CEID-Walk')
model_list.remove('NIH-Flu_ARIMA')
model_list

['CMU-TimeSeries',
 'CU-ensemble',
 'Flusight-baseline',
 'Flusight-ensemble',
 'GT-FluFNP',
 'JHU_IDD-CovidSP',
 'MIGHTE-Nsemble',
 'PSI-DICE',
 'SGroup-RandomForest',
 'SigSci-TSENS',
 'UMass-trends_ensemble',
 'UVAFluX-Ensemble',
 'VTSanghani-ExogModel',
 'VTSanghani-Transformer']

In [15]:
rmse_all = []
nrmse_all = []
model_all = []
mape_all = []
week_ahead = []
regions = []


In [16]:
for model in all_model_names:
    if model not in model_list:
        continue
    for i in range(1, 5):
        for region in all_regions:
            sample = df_newpoint[
                (df_newpoint["model"] == model)
                & (df_newpoint["ahead"] == i)
                & (df_newpoint["location_x"] == region)
            ]["value_x"].values
            target = df_newpoint[
                (df_newpoint["model"] == model)
                & (df_newpoint["ahead"] == i)
                & (df_newpoint["location_x"] == region)
            ]["value_y"].values
            rmse_all.append(rmse(sample, target))
            nrmse_all.append(norm_rmse(sample, target))

            #             Deal with inf values
            target = np.array([EPS if x == 0 else x for x in target]).reshape(
                (len(target), 1)
            )
            mape_all.append(mape(sample, target))
            model_all.append(model)
            week_ahead.append(i)
            regions.append(region)

/Users/srikarbalusu/Flusight-forecast-data/metrics.py:20: RuntimeWarning: Mean of empty slice.
  return np.sqrt(((predictions - targets) ** 2).mean())
/Users/srikarbalusu/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/srikarbalusu/Flusight-forecast-data/metrics.py:37: RuntimeWarning: Mean of empty slice.
  return np.sqrt(((predictions - targets) ** 2).mean())
/Users/srikarbalusu/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/srikarbalusu/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/srikarbalusu/Flusight-forecast-data/metrics.py:20: RuntimeWarning: Mean of empty slice.
  return np.sqrt(((predictions - targets) ** 2).mean())


In [17]:
df_point_scores = pd.DataFrame.from_dict(
    {
        "Model": model_all,
        "RMSE": rmse_all,
        "NRMSE": nrmse_all,
        "MAPE": mape_all,
        "Weeks ahead": week_ahead,
        "Location": regions,
    }
)


In [18]:
df_point_scores.head(50)

,Model,RMSE,NRMSE,MAPE,Weeks ahead,Location
0,CMU-TimeSeries,19.266665,0.385333,5.597656e+01,1,AK
1,CMU-TimeSeries,121.168206,1.223921,4.635940e+01,1,AL
2,CMU-TimeSeries,114.476349,0.315362,5.827857e+01,1,AR
3,CMU-TimeSeries,119.117750,0.206086,1.417179e+02,1,AZ
4,CMU-TimeSeries,558.270043,0.216384,1.143934e+02,1,CA
5,CMU-TimeSeries,29.078762,0.232630,7.073863e+01,1,CO
6,CMU-TimeSeries,42.011648,0.259331,7.607966e+01,1,CT
7,CMU-TimeSeries,19.245319,0.326192,7.915235e+01,1,DC
8,CMU-TimeSeries,16.345202,0.333576,1.786574e+02,1,DE
9,CMU-TimeSeries,187.851038,0.345950,4.289585e+01,1,FL


In [19]:
df_point_scores.to_csv('point_scores.csv')

In [20]:
# target is ground truth
df_quant = df_final[df_final['type']=='quantile']
df_quant

,model,forecast_week,ahead,location,type,quantile,value
1,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.010,53.337132
2,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.025,54.761291
3,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.050,77.427982
4,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.100,138.520775
5,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.150,158.317353
...,...,...,...,...,...,...,...
700437,VTSanghani-Transformer,202245,4,WY,quantile,0.850,21.307235
700438,VTSanghani-Transformer,202245,4,WY,quantile,0.900,26.563623
700439,VTSanghani-Transformer,202245,4,WY,quantile,0.950,31.820012
700440,VTSanghani-Transformer,202245,4,WY,quantile,0.975,34.448206


In [21]:
# norm_val = (df_quant['value']-df_quant['value'].min())/(df_quant['value'].max()-df_quant['value'].min())

norm_df_quant = df_quant.copy()
norm_df_quant['predicted_week'] = (
    norm_df_quant['forecast_week']+norm_df_quant['ahead']
)
norm_df_quant['predicted_week'] = norm_df_quant['predicted_week'].astype(str)
norm_df_quant = norm_df_quant[norm_df_quant['predicted_week']<=max_week] 
norm_df_quant

,model,forecast_week,ahead,location,type,quantile,value,predicted_week
1,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.010,53.337132,202243
2,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.025,54.761291,202243
3,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.050,77.427982,202243
4,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.100,138.520775,202243
5,CADPH-FluCAT_Ensemble,202242,1,CA,quantile,0.150,158.317353,202243
...,...,...,...,...,...,...,...,...
700389,VTSanghani-Transformer,202245,2,WY,quantile,0.850,14.873498,202247
700390,VTSanghani-Transformer,202245,2,WY,quantile,0.900,19.590285,202247
700391,VTSanghani-Transformer,202245,2,WY,quantile,0.950,24.307072,202247
700392,VTSanghani-Transformer,202245,2,WY,quantile,0.975,26.665465,202247


In [22]:
week_ahead = []
regions = []
crps_all = []
ls_all = []
model_all = []
cs_all = []

In [23]:
# Runtime warning - invalid value occurs during multiply -- ignore
import warnings
warnings.filterwarnings("ignore")

In [24]:
def pres_recall(mean, var, target, conf):
    """
    Fraction of GT points within the confidence interval
    Args:
        mean (np.ndarray): Mean of the distribution (N)
        var (np.ndarray): Variance of the distribution (N)
        target (np.ndarray): Target of the model (N)
        conf (float): Confidence level
    Returns:
        np.ndarray: Fraction of GT points within the confidence interval
    """
    low, high = conf_interval(mean, var, conf)
    truth = ((target > low) & (target < high)).astype("float32")
    if not isinstance(truth, np.ndarray):
        truth = np.array([truth])
    return truth.mean(-1)

In [25]:
def get_pr(pred, var, target, color="blue", label="FluFNP"):
    """
    Plot confidence and return Confidence score and AUC
    Args:
        pred (np.ndarray): Predictions of the model (N)
        var (np.ndarray): Variance of the distribution (N)
        target (np.ndarray): Target of the model (N)
        color (str): Color of the line
        label (str): Label of the model
    Returns:
        tuple: (Confidence score, AUC, fraction values)
    """ 
    pred_, var_, target_ = pred.squeeze(), var.squeeze(), target.squeeze()
    x = np.arange(0.05, 1.0, 0.01)
    y = np.array([pres_recall(pred_, var_, target_, c) for c in x])
    #     plt.plot(list(x) + [1.0], list(y) + [1.0], label=label, color=color)
    conf_score = np.abs(y - x).sum() * 0.01
    auc = y.sum() * 0.01
    return auc, conf_score, list(y) + [1.0]

In [26]:
# All models
count = 0
for model in all_model_names:
    if model not in model_list:
        continue
    print('Compiling scores of model ', model)
    print(f"Model {count}/{len(all_model_names)}")
    count += 1
    
#     All Weeks ahead
    for i in range(1, 5):
        print('Week ahead ', i)
        
#         All regions
        for region in all_regions:
            
#             Dataset with information about Ground truth ('value_y') and predictions ('value_x') 
            target = df_newpoint[ 
                                (df_newpoint['model']==model)
                                & (df_newpoint['ahead']==i)
                                & (df_newpoint['location_x']==region)
            ]
            
            norm_model = norm_df_quant[ 
                                (norm_df_quant['model']==model)
                                & (norm_df_quant['ahead']==i)  
                                & (norm_df_quant['location']==region)
            ]
            
            mean_ = []
            std_ = []
            var_ = []
            tg_vals = []
            pred_vals = []
            
            weeks = np.array(target['forecast_week'].drop_duplicates())
            if(len(weeks)!=0):
                for week in weeks:
    #                 Append point predictions
                    point_val = target[ (target['forecast_week']==week)][
                        'value_x'
                    ].values
                    mean_.append(point_val)
                    if(len(point_val)==0):
                        print(i, week, region, model)

    #                 Append point pred as predictions
                    predval = target[ (target['forecast_week']==week)][
                        'value_y'
                    ].values 
                    pred_vals.append(predval)
                
    #                     Append ground truth as target
                    tgval = target[ (target['forecast_week']==week)]['value_y'].values
                    tg_vals.append(tgval)

    #                 Find std from quantiles
                    b = norm_model[  
                                    (norm_model['forecast_week']==week) 
                                    & (norm_model['quantile']==0.75)
                    ]['value'].values
                    a = norm_model[  
                                    (norm_model['forecast_week']==week)
                                    & (norm_model['quantile']==0.25)
                    ]['value'].values
                    std = (b-a)/1.35
                
                    var = std**2
                    std_.append(std)
                    var_.append(var)
                for j in range(len(std_)): 
                    if len(std_[j]) == 0:
                        std_[j] = [0]
                std_ = np.array(std_)
                var_ = np.array(var_)
                pred_vals = np.array(pred_vals)
                mean_ = np.array(mean_)
                tg_vals = np.array(tg_vals)

                if(len(tg_vals)==0):
                    print(
                        "No target found for week ahead ",
                        i, 
                        " region ",
                        region,
                        "model ",
                        model
                    )

#             Calculate ls and crps
                if(0 in std_):
                    cr_target = np.array([EPS if x ==0 else x for x in tg_vals], dtype=object).reshape(
                        (len(tg_vals), 1)
                    )
                    cr = float(mape(pred_vals, cr_target))
                    ls = -10
#                     print(cr, ls)
                else:
                    cr = crps(mean_, std_, tg_vals)
                    ls = log_score(mean_, std_, tg_vals, window = 0.1)
                    if(ls<-10):
                        ls = -10
#                     print(cr, ls, "hi")  
                auc, cs, _ = get_pr(mean_, std_**2, tg_vals)

                
#                 if(ls<-10 or math.isnan(ls)):
#                     ls = -10
#                 elif(ls>10):
#                     ls = 10
#                 if(math.isnan(cr)):
#                     cr = 0
                    
                crps_all.append(cr)
                ls_all.append(ls)
#                 print(cs)
                cs_all.append(cs)
                
            else:
                crps_all.append(np.nan)
                ls_all.append(np.nan)
                cs_all.append(np.nan)
            week_ahead.append(i)
            regions.append(region)
            model_all.append(model)

Compiling scores of model  CMU-TimeSeries
Model 0/31
Week ahead  1
Week ahead  2
Week ahead  3
Week ahead  4
Compiling scores of model  CU-ensemble
Model 1/31
Week ahead  1
Week ahead  2
Week ahead  3
Week ahead  4
Compiling scores of model  Flusight-baseline
Model 2/31
Week ahead  1
Week ahead  2
Week ahead  3
Week ahead  4
Compiling scores of model  Flusight-ensemble
Model 3/31
Week ahead  1
Week ahead  2
Week ahead  3
Week ahead  4
Compiling scores of model  GT-FluFNP
Model 4/31
Week ahead  1
Week ahead  2
Week ahead  3
Week ahead  4
Compiling scores of model  JHU_IDD-CovidSP
Model 5/31
Week ahead  1
Week ahead  2
Week ahead  3
Week ahead  4
Compiling scores of model  MIGHTE-Nsemble
Model 6/31
Week ahead  1
Week ahead  2
Week ahead  3
Week ahead  4
Compiling scores of model  PSI-DICE
Model 7/31
Week ahead  1
Week ahead  2
Week ahead  3
Week ahead  4
Compiling scores of model  SGroup-RandomForest
Model 8/31
Week ahead  1
Week ahead  2
Week ahead  3
Week ahead  4
Compiling scores of m

In [27]:
df_spread_scores = pd.DataFrame.from_dict(
    {
        'Model':model_all,
        'Weeks ahead':week_ahead,
        'Location':regions,
        'LS':ls_all,
        'CRPS':crps_all,
        'CS':cs_all
    }
)

In [28]:
df_spread_scores.to_csv('spread_scores.csv')